In [ ]:
# (c) 2022-2023 RENCI/ESoP, Chapel Hill, NC

In [ ]:
import nltk
import numpy
import openpyxl
import pandas as pd
import string
import warnings

from ipyfilechooser import FileChooser
from nltk.sentiment import SentimentIntensityAnalyzer as sia
from openpyxl.utils.dataframe import dataframe_to_rows as df2r

pd.options.mode.chained_assignment = None # suppress warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

In [ ]:
pos_words1 = pd.read_table('dictionaries/green terminology.txt', header=None)
neg_words1 = pd.read_table('dictionaries/red terminology.txt', header=None)
neut_words = pd.read_table('dictionaries/white terminology.txt', header=None)

pos_words2 = pd.read_table('dictionaries/green terminology from literature.txt', header=None)
neg_words2 = pd.read_table('dictionaries/red terminology from literature.txt', header=None)

pos_words = pd.concat([pos_words1, pos_words2]).drop_duplicates()
neg_words = pd.concat([neg_words1, neg_words2]).drop_duplicates()

pos_list = []
pos_weight = []
for i in range(0, len(pos_words)):
    if pos_words.iloc[i, 1] != 0: # ignore entries with zero weight
        pos_list.append(pos_words.iloc[i, 0])
        pos_weight.append(pos_words.iloc[i, 1])

neg_list = []
neg_weight = []
for i in range(0, len(neg_words)):
    if neg_words.iloc[i, 1] != 0:
        neg_list.append(neg_words.iloc[i, 0])
        neg_weight.append(neg_words.iloc[i, 1])

neut_list = []
neut_weight = 0
for i in range(0, len(neut_words)):
    neut_list.append(neut_words.iloc[i, 0])

del pos_words
del pos_words1
del pos_words2
del neg_words
del neg_words1
del neg_words2
del neut_words

In [ ]:
nltk.download(['names', 'stopwords', 'state_union', 'averaged_perceptron_tagger', 'vader_lexicon'], quiet=True)

punct = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
stopwords = nltk.corpus.stopwords.words('english')

sii = sia()

approving = 'strengths of this experience' # literal to search in column headers
critical = 'better learning experience'
additional = 'Nothing further to add'

comment_type = [approving, critical, additional]
comment_weighting = [.25, .4, .35] # underweight approving comments, slightly overweight critical comments

e = .001

medium_cut = .05 # seems to be standard in literature for VADER compound score
high_cut = .12 # guess at reasonable value
adjustment_value = .03 + e # four net positive/negative statements imply high cutoff

very_pos_label = 'Very positive'
very_neg_label = 'Very negative'
pos_label = 'Positive'
neg_label = 'Negative'
neut_label = 'Neutral'

very_pos_score = high_cut + e
very_neg_score = -1 * very_pos_score
pos_score = medium_cut + e
neg_score = -1 * pos_score
neut_score = 0

overlong = 300 # words
lengthy = 200
brief = 15

n = numpy.nan
nl = 'null'

In [ ]:
def clear_punctuation(s):
    for t in s:
        if t in punct:
            s = s.replace(t, '')
    return s

def matches(s, t):
    if s.strip().lower() == t.strip().lower():
        return True
    return False

def biggest_val(l):
    if len(l) == 0:
        return None
    elif len(l) == 1:
        return l[0]
    elif len(l) == 2:
        if abs(l[0]) < abs(l[1]):
            return l[1]
        return l[0]
    else:
        return biggest_val([l[0], biggest_val(l[1:])])

def clean_data(df, cols): # prepare comments for analysis
    for j in cols:
        for i in range(len(df)):
            t = df.iloc[i, j]
            if isinstance(t, str):
                u = clear_punctuation(t.strip().lower())
                if matches(u, 'na'):
                    df.iloc[i, j] = None
                else:
                    df.iloc[i, j] = u
            else:
                df.iloc[i, j] = ''
    return df

def clean_vals(df, cols): # prepare numeric values for analysis
    for j in cols:
        for i in range(len(df)):
            v = df.iloc[i, j]
            try:
                if v is not None:
                    if not isinstance(v, int) and not isinstance(v, float) and not isinstance(v, complex):
                        if isinstance(v, str):
                            if v.isnumeric():
                                df.iloc[i, j] = float(v)
                            else:
                                df.iloc[i, j] = None
                        else:
                            df.iloc[i, j] = None
            except (TypeError, ValueError):
                df.iloc[i, j] = None
    return df

def get_col_index(df, s): # find which column holds a string
    for h in list(df.columns):
        if s in h:
            return df.columns.get_loc(h)

def clean_filename(s):
    f = ''.join(c for c in s.strip() if (c.isalnum() or c in '._- '))
    return f

def eval_weighting(w):
    if w < -1 * high_cut:
        return very_neg_label
    elif w < -1 * medium_cut:
        return neg_label
    elif w > high_cut:
        return very_pos_label
    elif w > medium_cut:
        return pos_label
    else:
        return neut_label

def is_approving_comment(i):
    if comment_type.index(approving) == i:
        return True
    return False

def is_critical_comment(i):
    if comment_type.index(critical) == i:
        return True
    return False

def is_additional_comment(i):
    if comment_type.index(additional) == i:
        return True
    return False

def is_overlong_comment(c):
    return (len(c.split()) > overlong)

def is_lengthy_comment(c):
    return (len(c.split()) > lengthy)

def is_brief_comment(c):
    return (len(c.split()) < brief)

def is_average_length_comment(c):
    return not (is_lengthy_comment(c) or is_brief_comment(c))

def is_very_pos_comment(c, t):
    if is_very_pos_score(biggest_val([get_lexical_semantic_score(c, t), get_sentiment_score(c, t)])):
        return True
    return False

def is_very_neg_comment(c, t):
    if is_very_neg_score(biggest_val([get_lexical_semantic_score(c, t), get_sentiment_score(c, t)])):
        return True
    return False

def is_pos_comment(c, t):
    if is_pos_score(biggest_val([get_lexical_semantic_score(c, t), get_sentiment_score(c, t)])):
        return True
    return False

def is_neg_comment(c, t):
    if is_neg_score(biggest_val([get_lexical_semantic_score(c, t), get_sentiment_score(c, t)])):
        return True
    return False

def is_at_all_pos_comment(c, t):
    if is_very_pos_comment(c, t):
        return True
    if is_pos_comment(c, t):
        return True
    return False

def is_at_all_neg_comment(c, t):
    if is_very_neg_comment(c, t):
        return True
    if is_neg_comment(c, t):
        return True
    return False

def is_neut_comment(c, t):
    for u in neut_list:
        if matches(u, c):
            return True
    if is_neut_score(biggest_val([get_lexical_semantic_score(c, t), get_sentiment_score(c, t)])):
        return True
    return False

def get_lexical_semantic_score(s, t): # s is the comment, t an indicator of type of comment
    w = 0
    for i, p in enumerate(pos_list):
        if p in s:
            w = min(1, w + adjustment_value * pos_weight[i] * comment_weighting[t])
    for j, n in enumerate(neg_list):
        if n in s:
            w = max(-1, w - adjustment_value * neg_weight[j] * comment_weighting[t])
    return w

def get_sentiment_score(s, t):
    d = .67 # temper scores as they tend to be high
    return sii.polarity_scores(s)['compound'] * comment_weighting[t] * d

def get_score(s, s_):
    if is_very_neg_comment(s, s_):
        return very_neg_score
    elif is_neg_comment(s, s_):
        return neg_score
    elif is_very_pos_comment(s, s_):
        return very_pos_score
    elif is_pos_comment(s, s_):
        return pos_score
    else:
        return neut_score

def is_very_pos_score(s):
    if matches(eval_weighting(s), very_pos_label):
        return True
    return False

def is_very_neg_score(s):
    if matches(eval_weighting(s), very_neg_label):
        return True
    return False

def is_pos_score(s):
    if matches(eval_weighting(s), pos_label):
        return True
    return False

def is_neg_score(s):
    if matches(eval_weighting(s), neg_label):
        return True
    return False

def is_neut_score(s):
    if matches(eval_weighting(s), neut_label):
        return True
    return False

In [ ]:
fc = FileChooser('data')
display(fc)

In [ ]:
df = pd.read_excel(fc.selected)

cols_of_interest = []

for i in range(0, len(comment_type)):
    cols_of_interest.append(get_col_index(df, comment_type[i]))

df = clean_data(df, cols_of_interest)

In [ ]:
# for training data
cols_of_interest.append(get_col_index(df, 'Rating'))

In [ ]:
# for test data
cols_of_interest.append(get_col_index(df, 'Quality of Interactions during Experience AVG'))
cols_of_interest.append(get_col_index(df, 'Quality of Preceptor/Preceptor Team AVG'))
cols_of_interest.append(get_col_index(df, 'Quality of Site AVG'))
df = clean_vals(df, cols_of_interest[-3:])

In [ ]:
df2 = df.iloc[:, cols_of_interest]

In [ ]:
# debug counts of cases
cases = []
counts = []

In [ ]:
def calculate_score(c):

    # consider, as needed, individually or in combination
    #  lexical/syntactic
    #  NL-based scores
    #  lengths of approving, critical, and additional comments

    a = ''
    x = ''
    d = ''

    a_ = 0
    x_ = 1
    d_ = 2

    al = []
    xl = []
    dl = []

    for i in range(0, len(c)):
        if is_approving_comment(i):
            if isinstance(c[i], str):
                a = clear_punctuation(c[i])
                a_ = i
                al.append(get_lexical_semantic_score(a, a_))
                al.append(get_sentiment_score(a, a_))
        elif is_critical_comment(i):
            if isinstance(c[i], str):
                x = clear_punctuation(c[i])
                x_ = i
                xl.append(get_lexical_semantic_score(x, x_))
                xl.append(get_sentiment_score(x, x_))
        elif is_additional_comment(i):
            if isinstance(c[i], str):
                d = clear_punctuation(c[i])
                d_ = i
                dl.append(get_lexical_semantic_score(d, d_))
                dl.append(get_sentiment_score(d, d_))

    j = nl

    # logic explained in logic/calculate_scores.xlsm

    if is_overlong_comment(x) and is_at_all_neg_comment(x, x_):
        j = 'n01'
        r = very_neg_score

    elif is_brief_comment(d) and is_very_neg_comment(d, d_):
        j = 'n02'
        r = get_score(d, d_) # very_neg_score

    elif is_average_length_comment(a) and (is_overlong_comment(x + d) or (is_lengthy_comment(x) and is_lengthy_comment(d))) and is_at_all_neg_comment(d, d_):
        j = 'n03'
        r = very_neg_score

    elif is_neut_comment(a, a_) and is_lengthy_comment(x) and is_at_all_neg_comment(x, x_) and is_neg_comment(d, d_):
        j = 'n04'
        r = very_neg_score

    elif not is_brief_comment(a) and is_overlong_comment(x) and not is_lengthy_comment(d):
        j = 'n05'
        r = neg_score

    elif is_brief_comment(a) and is_lengthy_comment(x) and is_lengthy_comment(d):
        j = 'n06'
        r = neg_score

    elif is_lengthy_comment(a) and is_neut_comment(a, a_) and is_at_all_neg_comment(x, x_) and is_at_all_neg_comment(d, d_):
        j = 'n07'
        r = neg_score

    elif is_lengthy_comment(a) and (is_overlong_comment(x + d) or (is_lengthy_comment(x) and is_lengthy_comment(d))):
        j = 'n08'
        r = neg_score

    elif is_pos_comment(a, a_) and is_brief_comment(a) and is_very_neg_comment(x, x_) and is_at_all_neg_comment(d, d_):
        j = 'n09'
        r = get_score(d, d_) # very_neg_score or neg_score

    elif not is_at_all_pos_comment(a, a_) and is_neut_comment(x, x_) and is_at_all_neg_comment(d, d_):
        j = 'n10'
        r = get_score(d, d_) # very_neg_score or neg_score

    elif is_neut_comment(a, a_) and not is_at_all_neg_comment(x, x_) and is_at_all_neg_comment(d, d_):
        j = 'n11'
        r = get_score(d, d_) # very_neg_score or neg_score

    elif is_neut_comment(a, a_) and is_at_all_neg_comment(x, x_) and not is_at_all_pos_comment(d, d_):
        j = 'n12'
        r = get_score(x, x_) # very_neg_score or neg_score

    elif is_at_all_neg_comment(a, a_) and (is_at_all_neg_comment(x, x_) or is_at_all_neg_comment(d, d_)):
        j = 'n13'
        r = get_score(a, a_) # very_neg_score or neg_score

    elif is_neg_comment(a, a_) and (is_overlong_comment(a) or is_lengthy_comment(a)) and is_neut_comment(x, x_):
        j = 'n14'
        r = neg_score

    elif is_brief_comment(x) and is_very_pos_comment(x, x_):
        j = 'p02'
        r = get_score(x, x_) # very_pos_score

    elif is_average_length_comment(x) and (is_overlong_comment(a + d) or (is_lengthy_comment(a) and is_lengthy_comment(d))) and is_at_all_pos_comment(d, d_):
        j = 'p03'
        r = very_pos_score

    elif is_lengthy_comment(a) and is_at_all_pos_comment(a, a_) and is_neut_comment(x, x_) and is_pos_comment(d, d_):
        j = 'p04'
        r = very_pos_score

    elif is_overlong_comment(a) and not is_brief_comment(x) and not is_lengthy_comment(d):
        j = 'p05'
        r = pos_score

    elif is_lengthy_comment(a) and is_brief_comment(x) and is_lengthy_comment(d):
        j = 'p06'
        r = pos_score

    elif is_at_all_pos_comment(a, a_) and is_lengthy_comment(x) and is_neut_comment(x, x_) and is_at_all_pos_comment(d, d_):
        j = 'p07'
        r = pos_score

    elif is_lengthy_comment(x) and (is_overlong_comment(a + d) or (is_lengthy_comment(a) and is_lengthy_comment(d))):
        j = 'p08'
        r = pos_score

    elif is_very_pos_comment(a, a_) and is_neg_comment(x, x_) and is_brief_comment(x) and is_at_all_pos_comment(d, d_):
        j = 'p09'
        r = get_score(d, d_) # very_pos_score or pos_score

    elif not is_at_all_pos_comment(a, a_) and is_neut_comment(x, x_) and is_at_all_pos_comment(d, d_):
        j = 'p10'
        r = get_score(d, d_) # very_pos_score or pos_score

    elif is_neut_comment(a, a_) and not is_at_all_neg_comment(x, x_) and is_at_all_pos_comment(d, d_):
        j = 'p11'
        r = get_score(d, d_) # very_pos_score or pos_score

    elif is_at_all_pos_comment(a, a_) and is_neut_comment(x, x_) and not is_at_all_neg_comment(d, d_):
        j = 'p12'
        r = get_score(a, a_) # very_pos_score or pos_score

    elif is_at_all_pos_comment(x, x_) and (is_at_all_pos_comment(a, a_) or is_at_all_pos_comment(d, d_)):
        j = 'p13'
        r = get_score(x, x_) # very_pos_score or pos_score

    elif is_pos_comment(x, x_) and (is_overlong_comment(x) or is_lengthy_comment(x)) and is_neut_comment(a, a_):
        j = 'p14'
        r = pos_score

    elif is_average_length_comment(a) and not is_lengthy_comment(x) and not is_lengthy_comment(d):

        if not is_at_all_pos_comment(a, a_) and is_very_neg_comment(x, x_) and not is_at_all_pos_comment(d, d_):
            j = 'n15'
            r = get_score(x, x_) # very_neg_score

        elif not is_at_all_pos_comment(a, a_) and is_neg_comment(x, x_) and is_very_neg_comment(d, d_):
            j = 'n16'
            r = get_score(d, d_) # very_neg_score

        elif not is_at_all_pos_comment(a, a_) and not is_at_all_pos_comment(x, x_) and is_very_neg_comment(d, d_):
            j = 'n17'
            r = get_score(d, d_) # very_neg_score

        elif is_neut_comment(a, a_) and is_at_all_neg_comment(x, x_) and is_at_all_neg_comment(d, d_):
            j = 'n18'
            r = very_neg_score

        elif is_at_all_pos_comment(a, a_) and is_very_neg_comment(x, x_) and is_very_neg_comment(d, d_):
            j = 'n19'
            r = neg_score

        elif is_pos_comment(a, a_) and is_at_all_neg_comment(x, x_) and is_at_all_neg_comment(d, d_):
            j = 'n20'
            r = neg_score

        elif is_pos_comment(a, a_) and is_neut_comment(x, x_) and is_very_neg_comment(d, d_):
            j = 'n21'
            r = neg_score

        elif is_pos_comment(a, a_) and is_very_neg_comment(x, x_) and is_neut_comment(d, d_):
            j = 'n22'
            r = neg_score

        elif not is_at_all_pos_comment(a, a_) and is_neg_comment(x, x_) and not is_at_all_pos_comment(d, d_):
            j = 'n23'
            r = get_score(x, x_) # neg_score

        elif is_neut_comment(a, a_) and is_at_all_neg_comment(x, x_) and is_neut_comment(d, d_):
            j = 'n24'
            r = get_score(x, x_) # very_neg_score or neg_score

        elif is_very_pos_comment(a, a_) and not is_at_all_neg_comment(x, x_) and not is_at_all_neg_comment(d, d_):
            j = 'p15'
            r = get_score(a, a_) # very_pos_score

        elif is_pos_comment(a, a_) and not is_at_all_neg_comment(x, x_) and is_very_pos_comment(d, d_):
            j = 'p16'
            r = get_score(d, d_) # very_pos_score

        elif not is_at_all_neg_comment(a, a_) and not is_at_all_neg_comment(x, x_) and is_very_pos_comment(d, d_):
            j = 'p17'
            r = get_score(d, d_) # very_pos_score

        elif is_at_all_pos_comment(a, a_) and is_neut_comment(x, x_) and is_at_all_pos_comment(d, d_):
            j = 'p18'
            r = very_pos_score

        elif is_very_pos_comment(a, a_) and is_at_all_neg_comment(x, x_) and is_very_pos_comment(d, d_):
            j = 'p19'
            r = pos_score

        elif is_at_all_pos_comment(a, a_) and is_neg_comment(x, x_) and is_at_all_pos_comment(d, d_):
            j = 'p20'
            r = pos_score

        elif is_neut_comment(a, a_) and is_neg_comment(x, x_) and is_very_pos_comment(d, d_):
            j = 'p21'
            r = pos_score

        elif is_very_pos_comment(a, a_) and is_neg_comment(x, x_) and is_neut_comment(d, d_):
            j = 'p22'
            r = pos_score

        elif is_pos_comment(a, a_) and not is_at_all_neg_comment(x, x_) and not is_at_all_neg_comment(d, d_):
            j = 'p23'
            r = get_score(a, a_) # pos_score

        elif is_at_all_pos_comment(a, a_) and is_neut_comment(x, x_) and is_neut_comment(d, d_):
            j = 'p24'
            r = get_score(a, a_) # very_pos_score or pos_score

        elif is_neut_comment(a, a_) and is_neut_comment(x, x_):
            j = '_99'
            r = get_score(d, d_)

    if matches(j, nl):
        j = '_00'
        r = neut_score # by default

    if cases == []:
        cases.append(j)
        counts.append(1)
    elif j in cases:
        counts[cases.index(j)] += 1
    else:
        cases.append(j)
        counts.append(1)

    return [r, biggest_val(al), biggest_val(xl), biggest_val(dl)]

In [ ]:
vals = []
evals = []
all_vals = []

for i in range(0, len(df2)): # loop for every row
    s = calculate_score(df2.iloc[i])
    vals.append(s[0])
    evals.append(eval_weighting(s[0])) # overall very negative, negative, neutral, positive, or very positive
    all_vals.append(s[1:]) # calcs for all comment types

In [ ]:
# debug counts of cases

assignments = []

for i, j in enumerate(cases):
    assignments.append(j + ': ' + str(counts[i]))

print(assignments)

del cases
del counts

In [ ]:
f = input('What is the name of the output file? ')
f = clean_filename(f)

In [ ]:
# for test data
df2['Overall Quality'] = df.iloc[:, cols_of_interest[-3:]].mean(axis=1, skipna=True)
df2[df2['Overall Quality'].eq('')] = n # may be missing data

In [ ]:
for i in range(0, len(comment_type)):
    df2['Lexical/Semantic Valuation--' + comment_type[i].lower()] = [v[i] for v in all_vals]

df2['Overall Valuation'] = vals
df2['Overall Evaluation'] = evals

wb = openpyxl.Workbook()
rows = df2r(df2, index=False)

for i, row in enumerate(rows, 1):
    for j, val in enumerate(row, 1):
         wb.active.cell(row=i, column=j, value=val)

wb.save('data/' + f + '.xlsx')

del rows
del wb